In [1]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
Cost_O=1
Cost_SC=1
M=40000
S=pd.read_excel(io=r'../data/fo_result_402.xlsx',sheet_name='page_1',usecols="B:OM").values.T
L=pd.read_excel(io=r'../data/fo_result_8.xlsx',sheet_name='page_1',usecols="B:I").values.T
L=L/100
MT=1./(pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="B").values.flatten())
Pr=pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="B").replace([0.6,0.66,0.72],[1.2*Cost_O,1.1*Cost_O,1*Cost_O]).values.flatten()

In [3]:
m = gp.Model("opt")
u0 = m.addVar(lb=0.0,vtype=GRB.INTEGER, name="u0")
v0 = m.addVar(lb=0.0,vtype=GRB.INTEGER, name="v0")
w0 = m.addVar(lb=0.0,vtype=GRB.INTEGER, name="w0")
xij = m.addMVar((402,24),lb=0.0,vtype=GRB.INTEGER,name='xij')
zijk = m.addMVar((402,24,8),lb=0.0,vtype=GRB.INTEGER,name='zijk')
u = m.addMVar(lb=0.0,shape=24,vtype=GRB.INTEGER, name='u')
v = m.addMVar(lb=0.0,shape=24,vtype=GRB.INTEGER, name='v')
w = m.addMVar(lb=0.0,shape=24,vtype=GRB.INTEGER, name='w')
absolu1=m.addMVar((402,24,8),lb=0.0,vtype=GRB.INTEGER,name='abs1')
absolu2=m.addMVar((402,24),lb=0.0,vtype=GRB.INTEGER,name='abs2')

In [4]:
obj1=sum(Pr @ xij)
m.setObjectiveN(obj1,index=0,name='obj1')
obj2=sum(sum(L[:,j] @ zijk[:,j,:].T) for j in range(24))
m.setObjectiveN(obj2,index=1,name='obj2')
obj3=Cost_SC*(24*(u0+v0+w0)+sum(sum((1-L[:,t]) @ zijk[:,t,:].T)-(u[t]+v[t]+w[t]) for j in range(1,25) for t in range(j)))
m.setObjectiveN(obj3,index=2,name='obj3')

In [5]:
c1=m.addConstr((S <= xij), name="c1")
c2=m.addConstr((zijk.sum(axis=2) == xij), name='c2')
c3=m.addConstr((zijk.sum(axis=0) <= 6000*np.ones((24,8))), name='c3')
c41=m.addConstrs((absolu1[:,:,k] >= zijk[:,:,k]/M for k in range(8)), name='c41')
c42=m.addConstrs((absolu1[:,:,k] <= zijk[:,:,k]/M+0.999 for k in range(8)), name='c42')
c43=m.addConstr((absolu2 >= xij/6000), name='c43')
c44=m.addConstr((absolu2 <= xij/6000+0.999), name='c44')
c4=m.addConstr((absolu1.sum(axis=2) == absolu2), name='c4')
c5=m.addConstr((u/0.6+v/0.66+w/0.72 == 28200*np.ones(24)), name='c5')
for j in range(1,25):
    for t in range(j):
        ue=sum((1-L[:,t]) @ zijk[np.where(Pr==1.2)[0].tolist(),t,:].T)-u[t]
        ve=sum((1-L[:,t]) @ zijk[np.where(Pr==1.1)[0].tolist(),t,:].T)-v[t]
        we=sum((1-L[:,t]) @ zijk[np.where(Pr==1.0)[0].tolist(),t,:].T)-w[t]
        m.addConstr(ue+u0 >= 0)
        m.addConstr(ve+v0 >= 0)
        m.addConstr(we+w0 >= 0)
    m.addConstr((ue+u0)/0.6+(ve+v0)/0.66+(we+w0)/0.72 >= 56400)

In [ ]:
m.Params.Threads=8
m.write("model.lp")
m.optimize()

In [ ]:
for i in range(m.NumObj):
	m.setParam(GRB.Param.ObjNumber, i)
	print(f"Obj {i+1} = {m.ObjNVal}")
x=np.zeros((402,24))
for i in range(402):
    for j in range(24):
        x[i,j]=m.getVarByName('xij['+str(i)+','+str(j)+']').X